# Apache Spark

Сегодня будет говорить про Apache Spark - более удобный фреймворк для обработки больших данных на базе Hadoop.

Ниже конфигурации для терраформа чтобы поднять себе кластер на Spark

In [ ]:
%%writefile common.tf

provider "azurerm" {
  version = "=2.40.0"
  features {}
}

resource "azurerm_resource_group" "naorlov_rg" {
  name = "naorlov-resource-group"
  location = "eastus"
}

resource "azurerm_virtual_network" "naorlov_vn" {
  name = "naorlov-vitrual-network"
  resource_group_name = azurerm_resource_group.naorlov_rg.name
  location = azurerm_resource_group.naorlov_rg.location
  address_space = ["10.0.0.0/16"]   # Пул адресов внутри сети
}

resource "azurerm_storage_account" "naorlov_sa" {
  name                     = "naorlovhdinsightstore"
  resource_group_name      = azurerm_resource_group.naorlov_rg.name
  location                 = azurerm_resource_group.naorlov_rg.location
  account_tier             = "Standard"
  account_replication_type = "LRS"
}

In [ ]:
%%writefile spark.tf

resource "azurerm_storage_container" "naorlov_spark_sc" {
  name                  = "naorlov-spark"
  storage_account_name  = azurerm_storage_account.naorlov_sa.name
  container_access_type = "private"
}

resource "azurerm_hdinsight_spark_cluster" "naorlov_sc" {
  name                = "naorlov-sparkcluster"
  resource_group_name = azurerm_resource_group.naorlov_rg.name
  location            = azurerm_resource_group.naorlov_rg.location
  cluster_version     = "4.0"
  tier                = "Standard"

  component_version {
    spark = "2.4"
  }

  gateway {
    enabled  = true
    username = "azureuser"
    password = "Password123!"
  }

  storage_account {
    storage_container_id = azurerm_storage_container.naorlov_spark_sc.id
    storage_account_key  = azurerm_storage_account.naorlov_sa.primary_access_key
    is_default           = true
  }

  roles {
    head_node {
      vm_size  = "A6"  # 2 cpu 14 ram
      username = "azureuser"
      password = "Password123!"
    }

    worker_node {
      vm_size               = "Standard_D12_V2" # 4 cpu 28 ram
      username              = "azureuser"
      password              = "Password123!"
      target_instance_count = 1
    }

    zookeeper_node {
      vm_size  = "Medium"
      username = "azureuser"
      password = "Password123!"
    }
  }
}


output "ssh_endpoint" {
    value = azurerm_hdinsight_spark_cluster.naorlov_sc.ssh_endpoint
}

output "https_endpoint" {
    value = azurerm_hdinsight_spark_cluster.naorlov_sc.https_endpoint
}

output "jupyter_endpoint" {
    value = "${azurerm_hdinsight_spark_cluster.naorlov_sc.https_endpoint}/jupyter"
}


Из интересных особенностей - этот кластер гораздо более дружелюбный к пользователю. Внутри него уже запущен специальный Jupyter Notebook сервис, к которому уже подключено все необходимое для работы с PySpark.

В текущей конфигурации terraform уже указан специальный output `jupyter_endpoint` - можно заметить, что устроен он достаточно просто. К основному `https_endpoint` вы просто добавляете `/jupyter`. Без него откроется админка Ambari, где можно посмотреть, как работают различные компоненты кластера (и например перезапустить сервис с Jupyter если он начал плохо работать).

Прокси для доступа к этим адресам не требуется - при открытии нужно лишь указать логин\пароль к машине.

Все ноутбуки, созданные через этот сервис автоматически сохраняются в облако WASB. Это означает, что если убить кластер, а потом создать новый и подключить к нему старое хранилище, то все ноутбуки будут сразу в рабочей директории. Конечно для супер надежности их можно сохранять и к себе на компьютер, но если вы не собираетесь удалять хранилище, то это достаточно удобный способ хранения ноутбуков.

В самом ноутбуке есть несколько особенностей:

Спарк ячейка автоматически пытаются подключить нужный контекст. Это означает, что в ноутбуке как будто бы уже определена переменная `spark`, которой можно пользоваться. Самой первой ячейкой в ноутбуке лучше сделать 

```python
sc = spark.sparkContext
```

В этот момент спарк прогреется и в переменной sc у вас уже будет лежать объект контекста

Ноутбук запущен на головной машине, однако запуск bash команд через восклицательный знак там не работает. Если хотите запускать bash, то используйте макрос `%%bash`


In [2]:
%%bash

pwd

/var/lib/jupyter


In [3]:
sp = spark.sparkContext  # Импортировать ничего не нужно!!

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1612336658629_0011,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


# Работаем с RDD

RDD - это базовый строительный блок для Spark. Спарк внимательно следит за тем, где лежат части RDD и как они были созданы. RDD по сути своей представляют упорядоченный набор записей. Большое число функций считают, что это пары ключ-значение (также как было в MapReduce), но на деле это может быть и произвольные данные.

RDD сами по себе неизменяемые. Можно лишь получить новый RDD, применяя различные операции к изначальному RDD.

Существуют два вида операций над RDD - Действия (actions) и Трансформации (transformations).

Трансформации не применяются сразу - они лишь записываются в пул примененных операций. Чтобы что-то действительно началось считаться, нужно применить уже действие - тогда все указанные транформации действительно начнут считаться на кластере.

Давайте сразу смотреть на примерах, чтобы стало понятно.

In [4]:
rdd = sc.parallelize(range(10))  # Создаем rdd из обычного списка

In [5]:
rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

In [6]:
rdd.collect()  # Получить значение всего RDD в память. Аккуратнее - если RDD большой, у вас просто лопнем питон

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
rdd.count()  # Считаем количество элементов в RDD

10

In [8]:
rdd.first()  # Берем только первый элемент

0

In [9]:
rdd.take(2)  # Берем первые N элементов

[0, 1]

In [10]:
rdd.mean()  # Считаем среднее по всем элементам. Важно, чтобы элементы внутри RDD поддерживали суммирование и деление

4.5

In [11]:
rdd = sc.parallelize(["biba", "kuka"])  # Можем положить и строки

In [12]:
rdd.collect()

['biba', 'kuka']

In [13]:
%%bash

hdfs dfs -rm -r /biba_and_kuka.txt || true

Deleted /biba_and_kuka.txt


21/02/03 11:58:02 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1
21/02/03 11:58:02 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Delete operation is: 121 ms with threads: 0


In [14]:
rdd.saveAsTextFile("/biba_and_kuka.txt")  # Сохраняем RDD в HDFS

In [29]:
%%bash

hdfs dfs -ls /biba_and_kuka.txt

Found 7 items
-rw-r--r--   1 livy supergroup          0 2021-02-03 08:48 /biba_and_kuka.txt/_SUCCESS
-rw-r--r--   1 livy supergroup          0 2021-02-03 08:48 /biba_and_kuka.txt/part-00000
-rw-r--r--   1 livy supergroup          0 2021-02-03 08:48 /biba_and_kuka.txt/part-00001
-rw-r--r--   1 livy supergroup          5 2021-02-03 08:48 /biba_and_kuka.txt/part-00002
-rw-r--r--   1 livy supergroup          0 2021-02-03 08:48 /biba_and_kuka.txt/part-00003
-rw-r--r--   1 livy supergroup          0 2021-02-03 08:48 /biba_and_kuka.txt/part-00004
-rw-r--r--   1 livy supergroup          5 2021-02-03 08:48 /biba_and_kuka.txt/part-00005


In [15]:
%%bash

hdfs dfs -cat /biba_and_kuka.txt/*

biba
kuka


Добавим теперь еще трансформации

In [16]:
rdd = sc.parallelize(range(10))  # Создаем rdd из обычного списка

In [20]:
# Создаем rdd в котором каждый элемент возведен в квадрат
# map работает примерно также как и map в MapReduce. 
# Разница - мы не обрабатываем блок самостоятельно, а пишем функцию для обработки ровно одной записи
squares = rdd.map(lambda x: x**2).map(lambda x: x + 1)

# ВАЖНО - на самом деле ничего считаться в этот момент не начало
# Мы лишь записали наше желание получить новый RDD и записали это желание в squares

In [21]:
squares.first() 

# Так как мы применили Action то вот теперь все трансформации запустились
# Но так как action требует только первую строку, то Spark оптимизировал вычисления
# он прочитал только первую строку и для нее вычислил значение

1

In [22]:
squares.collect()

[1, 2, 5, 10, 17, 26, 37, 50, 65, 82]

#### Начнем работать с данными

In [3]:
%%bash

# Заливаем данные в HDFS

hdfs dfs -mkdir /tweets_data

In [4]:
%%bash

for i in {1..13}
do 
    curl https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_$i.csv | tail -n +2 | hdfs dfs -put - /tweets_data/tweets_$i.csv
    echo "Finish $i"
done

Finish 1
Finish 2
Finish 3
Finish 4
Finish 5
Finish 6
Finish 7
Finish 8
Finish 9
Finish 10
Finish 11
Finish 12
Finish 13


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89.9M  100 89.9M    0     0  9615k      0  0:00:09  0:00:09 --:--:-- 14.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.0M  100 90.0M    0     0   9.9M      0  0:00:09  0:00:09 --:--:-- 13.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.0M  100 90.0M    0     0  8807k      0  0:00:10  0:00:10 --:--:-- 12.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.0M  100 90.0M    0     0  6302k      0  0:00:14  0:00:14 --:--:-- 24.9M
  % Total    % Received % Xferd  Average Speed   Tim

In [5]:
%%bash

hdfs dfs -ls /tweets_data

Found 13 items
-rw-r--r--   1 spark supergroup   94371561 2021-02-03 08:29 /tweets_data/tweets_1.csv
-rw-r--r--   1 spark supergroup   94371615 2021-02-03 08:31 /tweets_data/tweets_10.csv
-rw-r--r--   1 spark supergroup   94371552 2021-02-03 08:31 /tweets_data/tweets_11.csv
-rw-r--r--   1 spark supergroup   94371703 2021-02-03 08:31 /tweets_data/tweets_12.csv
-rw-r--r--   1 spark supergroup    8238864 2021-02-03 08:32 /tweets_data/tweets_13.csv
-rw-r--r--   1 spark supergroup   94371748 2021-02-03 08:30 /tweets_data/tweets_2.csv
-rw-r--r--   1 spark supergroup   94371796 2021-02-03 08:30 /tweets_data/tweets_3.csv
-rw-r--r--   1 spark supergroup   94371606 2021-02-03 08:30 /tweets_data/tweets_4.csv
-rw-r--r--   1 spark supergroup   94371616 2021-02-03 08:30 /tweets_data/tweets_5.csv
-rw-r--r--   1 spark supergroup   94371646 2021-02-03 08:30 /tweets_data/tweets_6.csv
-rw-r--r--   1 spark supergroup   94371711 2021-02-03 08:31 /tweets_data/tweets_7.csv
-rw-r--r--   1 spark supergroup   9

In [23]:
data = sp.textFile("/tweets_data/*")

In [24]:
data.first()

'906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on trial for corruption and you\'ve barely heard a peep from the mainstream media."" ~ @nedryun https://t.co/gh6g0D1oiC",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses/914580356430536707,https://twitter.com/10_gop/status/914580356430536707/video/1,,'

In [25]:
import csv

def extract_text(raw_string):
    parsed_line = next(csv.reader([raw_string]))
    text = parsed_line[2]
    return text

In [26]:
data.map(extract_text).first()

'"We have a sitting Democrat US Senator on trial for corruption and you\'ve barely heard a peep from the mainstream media." ~ @nedryun https://t.co/gh6g0D1oiC'

In [27]:
import re

def extract_words(text):
    pattern = re.compile(r"[a-z]+")
    result = []
    for match in pattern.finditer(text.lower()):
        word = match.group(0)
        result.append(word)
    return result

In [29]:
data.map(extract_text).map(extract_words).take(2)

[['we', 'have', 'a', 'sitting', 'democrat', 'us', 'senator', 'on', 'trial', 'for', 'corruption', 'and', 'you', 've', 'barely', 'heard', 'a', 'peep', 'from', 'the', 'mainstream', 'media', 'nedryun', 'https', 't', 'co', 'gh', 'g', 'd', 'oic'], ['marshawn', 'lynch', 'arrives', 'to', 'game', 'in', 'anti', 'trump', 'shirt', 'judging', 'by', 'his', 'sagging', 'pants', 'the', 'shirt', 'should', 'say', 'lynch', 'vs', 'belt', 'https', 't', 'co', 'mlh', 'i', 'lzz']]

In [30]:
data.map(extract_text).flatMap(extract_words).first()

'we'

In [31]:
data.map(extract_text).flatMap(extract_words).take(10)

['we', 'have', 'a', 'sitting', 'democrat', 'us', 'senator', 'on', 'trial', 'for']

Все трансформации вычисляются каждый раз с самого первого RDD. Чтобы уменьшить количество лишних вычислений можно закешировать временный результат. Тогда он будет по максимуму переиспользоваться.

In [32]:
words = data.map(extract_text).flatMap(extract_words).cache()

In [33]:
words.count()

41946754

In [34]:
words.count()

41946754

На моем запуске второй запуск `words.count()` работал 2 секунды вместо 17

#### Word count

Попробуем реализовать тот же алгоритм, что и для классического MapReduce

In [35]:
words.map(lambda x: (x, 1)).first()  # Строим пары ключ значение

('we', 1)

In [36]:
(
    words
    .map(lambda x: (x, 1))
    .groupByKey()  # Функция работает, только если RDD - это пары ключ-значение
    .take(2)
)

[('flufc', <pyspark.resultiterable.ResultIterable object at 0x7f182b3efeb8>), ('vyiey', <pyspark.resultiterable.ResultIterable object at 0x7f182b3ef550>)]

In [37]:
(
    words
    .map(lambda x: (x, 1))
    .groupByKey()  # Функция работает, только если RDD - это пары ключ-значение
    .map(lambda x: (x[0], sum(list(x[1]))))
    .take(10)
)

[('flufc', 1), ('ywqxtpzb', 1), ('fbzqvvo', 1), ('rxt', 37), ('jkmiles', 1), ('yngn', 4), ('barbarossa', 1), ('dxgxfhu', 1), ('desdemonalock', 2), ('lihua', 1)]

In [38]:
(
    words
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)  # Если уже готовая функция для reduce
    .take(10)
)

[('flufc', 1), ('vyiey', 1), ('fbzqvvo', 1), ('rxt', 37), ('jkmiles', 1), ('ywqxtpzb', 1), ('eunbb', 2), ('dxgxfhu', 1), ('desdemonalock', 2), ('ywhzvp', 1)]

In [39]:
(
    words
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])  # Сортируем по значению функции
)

[('t', 3015051), ('co', 2833375), ('https', 2454132), ('the', 591885), ('to', 589004), ('in', 457433), ('a', 412888), ('s', 397889), ('http', 375299), ('of', 350983)]

In [40]:
result_50 = (
    words
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(50, lambda x: -x[1])
)

stop_words = [word for word, _ in result_50]  # Предподсчитали стоп слова

In [41]:
stop_words

['t', 'co', 'https', 'the', 'to', 'in', 'a', 's', 'http', 'of', 'i', 'for', 'and', 'is', 'on', 'you', 'trump', 'news', 'it', 'with', 'at', 'that', 'this', 'rt', 'm', 'are', 'be', 'u', 'my', 'not', 'we', 'by', 'from', 'd', 'your', 'as', 'new', 'r', 'have', 'all', 'n', 'k', 'he', 'will', 'f', 'w', 'was', 'after', 'who', 'they']

In [42]:
(
    words
    .filter(lambda x: x not in stop_words)
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(50, lambda x: -x[1])
)

[('just', 63572), ('what', 62512), ('c', 62430), ('e', 62022), ('b', 61064), ('about', 60260), ('police', 59042), ('up', 58757), ('l', 58478), ('can', 58178), ('g', 57828), ('o', 57769), ('p', 57638), ('man', 57610), ('j', 57058), ('x', 56777), ('h', 56523), ('y', 56216), ('no', 55914), ('out', 55858), ('v', 55327), ('people', 55169), ('me', 54160), ('but', 53036), ('sports', 52223), ('so', 52082), ('if', 51593), ('obama', 50655), ('z', 50303), ('q', 50137), ('his', 49153), ('us', 47572), ('world', 47475), ('how', 46947), ('get', 46941), ('like', 46806), ('more', 46580), ('has', 45670), ('do', 45175), ('an', 44963), ('now', 44180), ('politics', 44031), ('don', 43979), ('when', 43894), ('amp', 43392), ('one', 43370), ('our', 43246), ('over', 42735), ('workout', 42437), ('black', 40133)]

Кроме базовых, есть еще и много продвинутых сложных функций
Например можем посчитать уникальные слова в датасете

Список всех можно смотреть в документации

https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

In [43]:
words.distinct().take(10)

['flufc', 'ywqxtpzb', 'fbzqvvo', 'rxt', 'jkmiles', 'yngn', 'barbarossa', 'dxgxfhu', 'desdemonalock', 'lihua']

In [44]:
words.distinct().count()

2831736

Однако иногда каких-то базовых примитивов может и не найтись. Например для RDD нет функции `limit` или около того.

Поэтому чтобы решить задачу top10 и сохранить это в HDFS нужно применить некоторую изобретательность

In [45]:
(
    words
    .filter(lambda x: x not in stop_words)
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .take(10)
)

[((63572, 'just'), 0), ((62512, 'what'), 1), ((62430, 'c'), 2), ((62022, 'e'), 3), ((61064, 'b'), 4), ((60260, 'about'), 5), ((59042, 'police'), 6), ((58757, 'up'), 7), ((58478, 'l'), 8), ((58178, 'can'), 9)]

In [47]:
(
    words
    .filter(lambda x: x not in stop_words)
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .filter(lambda x: x[1] < 10)
    .map(lambda x: x[0])
    .collect()
)

[(63572, 'just'), (62512, 'what'), (62430, 'c'), (62022, 'e'), (61064, 'b'), (60260, 'about'), (59042, 'police'), (58757, 'up'), (58478, 'l'), (58178, 'can')]

In [48]:
%%bash

hdfs dfs -rm -r /tweets_top10 || true

Deleted /tweets_top10


21/02/03 12:24:47 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1
21/02/03 12:24:47 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Delete operation is: 214 ms with threads: 0


In [49]:
(
    words
    .filter(lambda x: x not in stop_words)
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .filter(lambda x: x[1] < 10)
    .map(lambda x: x[0])
    .saveAsTextFile('/tweets_top10')
)

In [50]:
%%bash

hdfs dfs -cat /tweets_top10/*

(63572, 'just')
(62512, 'what')
(62430, 'c')
(62022, 'e')
(61064, 'b')
(60260, 'about')
(59042, 'police')
(58757, 'up')
(58478, 'l')
(58178, 'can')


#### Partitions

Под капотом Spark эксплуатирует примерно те же идеи, что и классический MapReduce. Это означает, что при необходимости сортировки, он разбивает ключи на группы и передает редюсерам на обработку только их часть.

На этот процесс также можно влиять. Это может позводить улучшить производительность программ, а также решить проблемы переполнения редюсеров.

In [51]:
words.getNumPartitions()

13

In [52]:
numbers = sc.parallelize(range(10))

In [53]:
numbers.glom().collect()  # Получаем доступ до данных в каждей партиции

[[0], [1, 2], [3, 4], [5], [6, 7], [8, 9]]

In [54]:
squares = numbers.map(lambda x: (x, x**2))

Операции изменения партиций предполагают наличие ключа, поэтому вначале преобразуем данные к виду ключ-значение

In [55]:
squares.partitionBy(2).glom().collect()

[[(0, 0), (4, 16), (6, 36), (2, 4), (8, 64)], [(1, 1), (5, 25), (9, 81), (3, 9), (7, 49)]]

In [57]:
squares.partitionBy(15).glom().collect()

[[(0, 0)], [(1, 1)], [(2, 4)], [(3, 9)], [(4, 16)], [(5, 25)], [(6, 36)], [(7, 49)], [(8, 64)], [(9, 81)], [], [], [], [], []]

In [58]:
def custom_partitioner(value):
    return value % 3

In [59]:
squares.partitionBy(3, custom_partitioner).glom().collect()

[[(0, 0), (3, 9), (6, 36), (9, 81)], [(1, 1), (4, 16), (7, 49)], [(2, 4), (5, 25), (8, 64)]]

Таким образом можно выбирать более удачные способы разбиения и например увеличивать количество редюсеров под вашу задачу.

Или наоборот, уменьшать количество количество партиций, если они избыточны. Например вы отфильтровали гигантский датасет и теперь вам больше не требуется такое гигантское количество партиций для работы.

Для этого можно использовать и `repartition` как делали выше, однако этот метод запустит пересортировку вообще всего RDD, что дорого и излишне. Чтобы так не было, можно использовать функцию `coalesce` - она просто схлопнуть вместе те партиции, котороые уже находятся на одной машине, что значительно уменьшит количество лишних телодвижений.

In [60]:
squares.glom().collect()

[[(0, 0)], [(1, 1), (2, 4)], [(3, 9), (4, 16)], [(5, 25)], [(6, 36), (7, 49)], [(8, 64), (9, 81)]]

In [61]:
squares.filter(lambda x: x[0] >= 7).glom().collect()

[[], [], [], [], [(7, 49)], [(8, 64), (9, 81)]]

In [62]:
squares.filter(lambda x: x[0] >= 7).coalesce(2).glom().collect()

[[], [(7, 49), (8, 64), (9, 81)]]

#### DataFrame и SQL

Уже текущий набор функций - это большой шаг вперед относительно классического MapReduce. Однако на этом плюшки Spark не заканчиваются. Разработчики пошли дальше и начали внедрять еще более высокоуровневый интерфейс для работы с данными, который может сильно упростить жизнь разработчикам.

DataFrame - это модель таблицы, построенная поверх RDD. О ней можно думать как о Pandas на стероидах.

In [63]:
rdd = sc.parallelize([("a", 1), ("a", 2), ("b", 3), ("b", 4)])
rdd.collect()

[('a', 1), ('a', 2), ('b', 3), ('b', 4)]

In [64]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [65]:
df = se.createDataFrame(rdd)
df.printSchema()
df.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+---+---+
| _1| _2|
+---+---+
|  a|  1|
|  a|  2|
|  b|  3|
|  b|  4|
+---+---+

In [66]:
df = se.createDataFrame(
    rdd.map(lambda x: Row(pipa=x[0], pupa=x[1]))
)
df.printSchema()
df.show()

root
 |-- pipa: string (nullable = true)
 |-- pupa: long (nullable = true)

+----+----+
|pipa|pupa|
+----+----+
|   a|   1|
|   a|   2|
|   b|   3|
|   b|   4|
+----+----+

Для удобства есть встроенные функции конвертации в pandas и оттуда

In [67]:
pandas_df = df.toPandas()
pandas_df

  pipa  pupa
0    a     1
1    a     2
2    b     3
3    b     4

In [68]:
df = se.createDataFrame(pandas_df)
df.printSchema()
df.show()

root
 |-- pipa: string (nullable = true)
 |-- pupa: long (nullable = true)

+----+----+
|pipa|pupa|
+----+----+
|   a|   1|
|   a|   2|
|   b|   3|
|   b|   4|
+----+----+

Есть специальные функции, которые умеют работать с популярными форматами хранения таблиц, и строить их в HDFS.

Прочтем нашу таблицу через DataFrame

In [69]:
df = spark.read.csv('/tweets_data/*', header=False, inferSchema=True)

In [70]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)

In [71]:
columns = [
    'external_author_id',
    'author',
    'content',
    'region',
    'language',
    'publish_date',
    'harvested_date',
    'following',
    'followers',
    'updates',
    'post_type',
    'account_type',
    'retweet',
    'account_category',
    'new_june_2018',
    'alt_external_id',
    'tweet_id',
    'article_url',
    'tco1_step1',
    'tco2_step1',
    'tco3_step1'
]
df = df.toDF(*columns)
df.printSchema()

root
 |-- external_author_id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- content: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- publish_date: string (nullable = true)
 |-- harvested_date: string (nullable = true)
 |-- following: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- updates: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- retweet: string (nullable = true)
 |-- account_category: string (nullable = true)
 |-- new_june_2018: string (nullable = true)
 |-- alt_external_id: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- article_url: string (nullable = true)
 |-- tco1_step1: string (nullable = true)
 |-- tco2_step1: string (nullable = true)
 |-- tco3_step1: string (nullable = true)

In [72]:
df.show()

+------------------+---------------+--------------------+--------------------+-------------+---------------+---------------+--------------+---------+-------+---------+------------+-------+----------------+-------------+---------------+------------------+--------------------+--------------------+--------------------+----------+
|external_author_id|         author|             content|              region|     language|   publish_date| harvested_date|     following|followers|updates|post_type|account_type|retweet|account_category|new_june_2018|alt_external_id|          tweet_id|         article_url|          tco1_step1|          tco2_step1|tco3_step1|
+------------------+---------------+--------------------+--------------------+-------------+---------------+---------------+--------------+---------+-------+---------+------------+-------+----------------+-------------+---------------+------------------+--------------------+--------------------+--------------------+----------+
|        1647

In [73]:
df[['author', 'content']].show()

+---------------+--------------------+
|         author|             content|
+---------------+--------------------+
|CARRIETHORNTHON|New Study Reveals...|
|CARRIETHORNTHON|Lindsey Graham ha...|
|CARRIETHORNTHON|. @LindseyGrahamS...|
|CARRIETHORNTHON|2016 Power Index:...|
|CARRIETHORNTHON|I self identify a...|
|CARRIETHORNTHON|.@UW varsity eigh...|
|CARRIETHORNTHON|Since the 1980s a...|
|CARRIETHORNTHON|Records: Wife Sus...|
|CARRIETHORNTHON|"""It will create...|
|CARRIETHORNTHON|The good news abo...|
|CARRIETHORNTHON|"""Culture fit"" ...|
|CARRIETHORNTHON|Things to Know Ab...|
|CARRIETHORNTHON|This is what happ...|
|CARRIETHORNTHON|#FloridaMan throw...|
|CARRIETHORNTHON|Report: DHS 'Red-...|
|CARRIETHORNTHON|More proof that T...|
|CARRIETHORNTHON|World Naked Bike ...|
|CARRIETHORNTHON|#Business — New @...|
|CARRIETHORNTHON|Pre-Game warm up ...|
|CARRIETHORNTHON|#News — Retired #...|
+---------------+--------------------+
only showing top 20 rows

In [74]:
df.registerTempTable('tweets')  # Регистрируем как временную таблицу для SQL

In [75]:
se.sql("""
    SELECT author, content, followers
    FROM tweets
    WHERE followers > 100
    LIMIT 10
""").show()

+---------------+--------------------+---------+
|         author|             content|followers|
+---------------+--------------------+---------+
|CARRIETHORNTHON|New Study Reveals...|      207|
|CARRIETHORNTHON|Lindsey Graham ha...|      207|
|CARRIETHORNTHON|. @LindseyGrahamS...|      207|
|CARRIETHORNTHON|2016 Power Index:...|      207|
|CARRIETHORNTHON|I self identify a...|      207|
|CARRIETHORNTHON|.@UW varsity eigh...|      207|
|CARRIETHORNTHON|Since the 1980s a...|      207|
|CARRIETHORNTHON|Records: Wife Sus...|      207|
|CARRIETHORNTHON|"""It will create...|      207|
|CARRIETHORNTHON|The good news abo...|      207|
+---------------+--------------------+---------+

In [76]:
se.sql("""
    SELECT language, count(*) as tw_count
    FROM tweets
    WHERE followers > 100
    GROUP BY language
""").show()

+------------------+--------+
|          language|tw_count|
+------------------+--------+
|              Urdu|      49|
|          Malaysia|      27|
|           Turkish|     373|
|              Iraq|     275|
|           Germany|     190|
|       Afghanistan|      35|
|           Kannada|       1|
|             Malay|     216|
|           Finnish|     520|
|              Thai|      33|
|            France|      11|
|         Icelandic|     455|
|            Pushto|     309|
|            Somali|     238|
|        Indonesian|     155|
|LANGUAGE UNDEFINED|    8143|
|              null|     157|
|         Ukrainian|   34620|
|Tagalog (Filipino)|     215|
|     United States|   14809|
+------------------+--------+
only showing top 20 rows

In [77]:
top5_lang = se.sql("""
    SELECT language, count(*) as tw_count
    FROM tweets
    WHERE followers > 100
    GROUP BY language
    ORDER BY tw_count DESC
    LIMIT 5
""")
top5_lang.show()

+-------------+--------+
|     language|tw_count|
+-------------+--------+
|      English| 1913019|
|      Russian|  546315|
|       German|   61941|
|    Ukrainian|   34620|
|United States|   14809|
+-------------+--------+

In [78]:
only_langs_df = se.sql("""
    SELECT language
    FROM (
        SELECT language, count(*) as tw_count
        FROM tweets
        WHERE followers > 100
        GROUP BY language
        ORDER BY tw_count DESC
        LIMIT 5
    )
""")
only_langs_df.show()

+-------------+
|     language|
+-------------+
|      English|
|      Russian|
|       German|
|    Ukrainian|
|United States|
+-------------+

In [79]:
only_langs_df.registerTempTable('languages')

In [80]:
se.sql("""
    SELECT author, language
    FROM tweets
    WHERE language in (SELECT * FROM languages)
    LIMIT 10
""").show()

+---------------+--------+
|         author|language|
+---------------+--------+
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
+---------------+--------+

In [81]:
se.sql("""
    SELECT author, t.language
    FROM tweets t
        inner join languages l on l.language = t.language
    LIMIT 10
""").show()

+---------------+--------+
|         author|language|
+---------------+--------+
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
+---------------+--------+

In [83]:
# Из под датафрейма всегда можно вынуть RDD и работать напрямую уже с ним

top5_lang.rdd.map(lambda x: x.language.upper()).collect()

['ENGLISH', 'RUSSIAN', 'GERMAN', 'UKRAINIAN', 'UNITED STATES']

In [85]:
top5_lang.collect()

[Row(language='English', tw_count=1913019), Row(language='Russian', tw_count=546315), Row(language='German', tw_count=61941), Row(language='Ukrainian', tw_count=34620), Row(language='United States', tw_count=14809)]